In [1]:
import pandas as pd
from pathlib import Path
from src.factiva_api.extraction import avro_to_dataframe
avro_dir = Path("data_avro") 
dfs = [avro_to_dataframe(p) for p in avro_dir.glob("*.avro")]
pd.concat(dfs, ignore_index=True).to_csv("all_articles.csv", index=False)

2025-08-07 13:57:04 [INFO] Read 32083 records from data_avro/part-000000000000.avro. Converting to DataFrame.


In [2]:
df = pd.read_csv("all_articles.csv")

In [11]:
df =df[:10]

In [5]:
len(df)

5000

In [12]:
df.to_csv("all_articles_10.csv", index=False)

In [10]:
df.columns

Index(['source_name', 'dateline', 'ingestion_datetime', 'currency_codes',
       'company_codes_association_ticker_exchange', 'title', 'snippet',
       'company_codes_lineage_ticker_exchange', 'an',
       'company_codes_occur_ticker_exchange', 'availability_datetime',
       'copyright', 'modification_date', 'industry_codes', 'language_code',
       'company_codes_ticker_exchange', 'region_of_origin',
       'company_codes_relevance', 'word_count', 'company_codes_about',
       'byline', 'person_codes', 'company_codes_lineage', 'body',
       'publication_datetime', 'publication_date', 'subject_codes',
       'publisher_name', 'source_code', 'credit', 'art',
       'company_codes_association', 'document_type', 'modification_datetime',
       'company_codes', 'action', 'region_codes', 'market_index_codes',
       'company_codes_about_ticker_exchange', 'company_codes_occur', 'section',
       'company_codes_relevance_ticker_exchange'],
      dtype='object')

In [3]:
from llm_utils.factory import LLMChainFactory

factory = LLMChainFactory(model_name="o4-mini")

In [ ]:
chain = factory.build_llm_with_web_search_tool("o3")

In [17]:
chain.invoke("What is the results of Arsenal vs Villareal friendly match on 2025-08-06?")

2025-08-08 08:05:25 [INFO] HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


AIMessage(content=[{'type': 'text', 'text': 'Arsenal faced Villarreal in a pre-season friendly at the Emirates Stadium on August 6, 2025, resulting in a 3-2 defeat for the Gunners. ([skysports.com](https://www.skysports.com/football/arsenal-vs-villarreal/report/537987?utm_source=openai))\n\n**Match Summary:**\n\n- **16th Minute:** Former Arsenal winger Nicolas Pépé opened the scoring for Villarreal, capitalizing on a defensive lapse to net from close range. ([outlookindia.com](https://www.outlookindia.com/sports/football/arsenal-vs-villarreal-match-report-pre-season-friendlies-2025-nicolas-pepe?utm_source=openai))\n\n- **33rd Minute:** Karl Etta Eyong doubled Villarreal\'s lead, converting a rebound after David Raya parried Gerard Moreno\'s shot. ([outlookindia.com](https://www.outlookindia.com/sports/football/arsenal-vs-villarreal-match-report-pre-season-friendlies-2025-nicolas-pepe?utm_source=openai))\n\n- **36th Minute:** Arsenal responded swiftly as Christian Nørgaard headed in a c

In [26]:
from pydantic import BaseModel, Field

In [27]:
class SearchResult(BaseModel):
    game_titles: list[str] = Field(
        description="List of video game titles")
    game_release_dates: list[str] = Field(
        description="List of video game release dates")

In [28]:
chain = factory.build_search_runnable_with_structured_output_debug(SearchResult, "o3")
chain.invoke("What Death Stranding games are available as of August 2025?, what are their release dates?")

AttributeError: 'LLMChainFactory' object has no attribute 'build_search_runnable_with_structured_output_debug'

In [4]:
from llm_utils.factory import LLMChainFactory
factory = LLMChainFactory(model_name="o4-mini", provider="azure")

In [3]:
factory.llm.invoke("What is the results of Arsenal vs Villareal friendly match on 2025-08-06?")

NameError: name 'factory' is not defined

In [5]:
from pydantic import BaseModel, Field

class GamesSearchResult(BaseModel):
    game_titles: list[str] = Field(
        description="List of video game titles")
    game_release_dates: list[str] = Field(
        description="List of video game release dates")
    
chain = factory.build_search_runnable_with_structured_output(GamesSearchResult, "gpt-5")
chain.invoke("What Death Stranding games are available as of August 2025?, what are their release dates?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


AIMessage(content=[{'type': 'text', 'text': '{"game_titles":["Death Stranding","Death Stranding: Director’s Cut","Death Stranding 2: On the Beach"],"game_release_dates":["Death Stranding — PS4: November 8, 2019; Windows (Steam/Epic): July 14, 2020. \ue200cite\ue202turn1search0\ue202turn6search0\ue201","Death Stranding: Director’s Cut — PS5: September 24, 2021; Windows (Steam/Epic): March 30, 2022; iPhone 15 Pro/iPad/Mac (App Store): January 30, 2024; Xbox Series X|S: November 7, 2024; Amazon Luna: November 7, 2024. \ue200cite\ue202turn0search0\ue202turn5search0\ue202turn3search0\ue202turn4search0\ue201","Death Stranding 2: On the Beach — PS5: June 26, 2025. \ue200cite\ue202turn8search0\ue201"]}', 'annotations': []}], additional_kwargs={'reasoning': {'id': 'rs_689a05deea9c81a18f1136a604b0866d0d4aea25ad3a231c', 'summary': [], 'type': 'reasoning'}, 'tool_outputs': [{'id': 'ws_689a05a718b881a1be1e1ed0956ca84e0d4aea25ad3a231c', 'action': {'query': 'Death Stranding original release date PS4'

In [2]:
import pandas as pd
import asyncio
from pathlib import Path
from langchain_core.prompts import PromptTemplate
from tqdm.asyncio import tqdm

from llm_utils.factory import LLMChainFactory
from src.schemas.models import CarveOutIdentificationSummary, SearchCarveOutIdentificationSummary
from src.schemas.prompts import IDENTIFICATION_PROMPT_TEMPLATE, IDENTIFICATION_SEARCH_PROMPT_TEMPLATE, business_request

# --- 1. Prepare input -------------------------------------------------------
def prepare_input(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv("/Users/sylvestrsemesko/Downloads/final_analysis_results(Sheet1) (2).csv")
    df = df[df["is_relevant"] & df["is_about_carve_out"]]

    # rename to match IDENTIFICATION_PROMPT_TEMPLATE variables
    df = df.rename(columns={
        "segments": "article_fragment",
        "company_names": "companies",
        "modification_date": "date",
    })

    # create placeholder fields expected by the prompt
    for col in ["title", "subsidiary_company_code", "carve_out_stage"]:
        if col not in df.columns:
            df[col] = ""
    
    # Add reasoning field - map from existing column or create empty
    if "short_reasoning" in df.columns:
        df["reasoning"] = df["short_reasoning"]
    else:
        df["reasoning"] = ""
    
    # Add original index to preserve row order
    df = df.reset_index(drop=True)
    df['original_index'] = df.index
    
    df = df[:600]

    return df

# --- 2. Asynchronous LLM step -----------------------------------------------
async def identify_carveouts(df: pd.DataFrame, concurrency: int = 10) -> pd.DataFrame:
    factory = LLMChainFactory(model_name="gpt-5", provider="openai", streaming=False)
    
    chain = factory.runnable_with_pydantic(
        template=IDENTIFICATION_PROMPT_TEMPLATE,
        pydantic_model=CarveOutIdentificationSummary,
        llm=factory.llm
    )

    sem = asyncio.Semaphore(concurrency)

    async def call_llm(row, pbar):
        async with sem:
            try:
                # Map DataFrame columns to template variables exactly as they appear in the prompt
                template_data = {
                    "business_request": business_request,
                    "source_name": row.get("source_name", ""),
                    "title": row.get("title", ""),
                    "date": row.get("date", ""),
                    "article_fragment": row.get("article_fragment", ""),
                    "companies": row.get("companies", ""),
                    "company_codes": row.get("company_codes", ""),
                    "target_company_code": row.get("target_company_code", ""),
                    "subsidiary_company_code": row.get("subsidiary_company_code", ""),
                    "carve_out_stage": row.get("carve_out_stage", ""),
                    "reasoning": row.get("reasoning", "")
                }
                
                resp = await chain.ainvoke(template_data)
                
                # For runnable_with_pydantic, resp is directly the Pydantic model
                parsed_data = resp.model_dump()
                
                # Add the original index to track which input row this came from
                parsed_data["original_index"] = row["original_index"]
                
                return parsed_data
            finally:
                pbar.update(1)

    rows = df.to_dict(orient="records")
    
    # Create progress bar
    with tqdm(total=len(rows), desc="Processing carve-out identification") as pbar:
        tasks = [asyncio.create_task(call_llm(row, pbar)) for row in rows]
        outputs = await asyncio.gather(*tasks)
    
    # Create DataFrame from LLM outputs
    df_llm_results = pd.DataFrame(outputs)
    
    # Merge with original input data
    df_combined = df.merge(df_llm_results, on='original_index', how='left')
    
    # Drop the helper column
    df_combined = df_combined.drop('original_index', axis=1)
    
    return df_combined


# --- 3. Execute in Jupyter ---------------------------------------------------
csv_path = "path/to/previous_results.csv"  # adjust as needed

# Step 1: Initial carve-out identification
df_input = prepare_input(csv_path)
print(f"Processing {len(df_input)} rows...")

df_output = await identify_carveouts(df_input, concurrency=50)
df_output.to_csv("outputs/carveout_results.csv", index=False)

print("Carve-out identification completed!")
print(f"Output columns: {list(df_output.columns)}")
df_output.head()

Processing 600 rows...


Processing carve-out identification:   0%|          | 0/600 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTT

Carve-out identification completed!
Output columns: ['source_name', 'source_code', 'industry_codes', 'company_codes', 'subject_codes', 'publication_datetime', 'ingestion_datetime', 'date', 'region_codes', 'an', 'action', 'companies', 'industries', 'article_fragment', 'segment_word_count', 'is_about_carve_out', 'target_company_code', 'is_relevant', 'short_reasoning', 'original_company_codes', 'original_companies', 'index', 'target_company_x', 'title', 'subsidiary_company_code', 'carve_out_stage', 'reasoning', 'target_company_y', 'financial_group', 'financial_group_hq', 'group_industry', 'group_vertical', 'potential_disposal', 'potential_disposal_company', 'potential_disposal_country', 'potential_disposal_industry', 'potential_disposal_vertical', 'relevant', 'interest_score', 'rationale', 'article_quote']


,source_name,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,date,region_codes,an,...,group_vertical,potential_disposal,potential_disposal_company,potential_disposal_country,potential_disposal_industry,potential_disposal_vertical,relevant,interest_score,rationale,article_quote
0,Acquisdata Global Industry SnapShot,ACQIND,",i3441,i831,i83107,iblock,ifinal,ifmsoft,iinv,...","aencre,afsvcg,alibab,allpau,altosv,amexpr,amzc...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,gcat,g...",2025-04-21 00:00:00,2025-04-25 10:06:58,2025-04-26 07:33:20.526,",eurz,namz,uk,usa,weurz,",ACQIND0020250425el4l0000b,...,Banking,British payments business (creation of a stand...,Unknown,GB,Technology & Payments,Digital Payments,False,0.10,Barclays is carving out its British payments b...,Barclays… has agreed terms for Brookfield Asse...
1,Acquisdata Global Industry SnapShot,ACQIND,",i3441,i831,i83107,iblock,ifinal,ifmsoft,iinv,...","aencre,afsvcg,alibab,allpau,altosv,amexpr,amzc...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,gcat,g...",2025-04-21 00:00:00,2025-04-25 10:06:58,2025-04-26 07:33:20.526,",eurz,namz,uk,usa,weurz,",ACQIND0020250425el4l0000b,...,Digital Payments,Issuer Solutions business,Unknown,Unknown,Technology & Payments,Digital Payments,False,0.85,The company is reshaping its portfolio alongsi...,Unknown
2,Acquisdata Global Industry SnapShot,ACQIND,",i3441,i831,i83107,iblock,ifinal,ifmsoft,iinv,...","aencre,afsvcg,alibab,allpau,altosv,amexpr,amzc...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,gcat,g...",2025-04-21 00:00:00,2025-04-25 10:06:58,2025-04-26 07:33:20.526,",eurz,namz,uk,usa,weurz,",ACQIND0020250425el4l0000b,...,Digital Payments,Issuer Solutions business (issuer processing/s...,Issuer Solutions business,US,Technology & Payments,Digital Payments (Issuer Processing),False,0.20,The company is divesting a defined business un...,Global Payments Inc. ... announced definitive ...
3,Acquisdata Global Industry SnapShot,ACQIND,",i3441,i831,i83107,iblock,ifinal,ifmsoft,iinv,...","aencre,afsvcg,alibab,allpau,altosv,amexpr,amzc...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,gcat,g...",2025-04-21 00:00:00,2025-04-25 10:06:58,2025-04-26 07:33:20.526,",eurz,namz,uk,usa,weurz,",ACQIND0020250425el4l0000b,...,Consumer Finance,Unknown,Unknown,Unknown,Unknown,Unknown,False,0.20,"Only a leadership change was announced, with n...","On 23 March 2025, AEON announced appointment o..."
4,Acquisdata Global Industry SnapShot,ACQIND,",i3441,i831,i83107,iblock,ifinal,ifmsoft,iinv,...","aencre,afsvcg,alibab,allpau,altosv,amexpr,amzc...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,gcat,g...",2025-04-21 00:00:00,2025-04-25 10:06:58,2025-04-26 07:33:20.526,",eurz,namz,uk,usa,weurz,",ACQIND0020250425el4l0000b,...,Digital Payments,Separation and possible carve-out of credit ca...,Unknown,CN,Financial Services,Consumer Lending,False,0.20,Regulatory-driven restructuring may force Ant ...,"Ant would also cut any linkage between Alipay,..."


In [3]:
df_output

,source_name,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,date,region_codes,an,...,group_vertical,potential_disposal,potential_disposal_company,potential_disposal_country,potential_disposal_industry,potential_disposal_vertical,relevant,interest_score,rationale,article_quote
0,Acquisdata Global Industry SnapShot,ACQIND,",i3441,i831,i83107,iblock,ifinal,ifmsoft,iinv,...","aencre,afsvcg,alibab,allpau,altosv,amexpr,amzc...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,gcat,g...",2025-04-21 00:00:00,2025-04-25 10:06:58,2025-04-26 07:33:20.526,",eurz,namz,uk,usa,weurz,",ACQIND0020250425el4l0000b,...,Banking,British payments business (creation of a stand...,Unknown,GB,Technology & Payments,Digital Payments,False,0.10,Barclays is carving out its British payments b...,Barclays… has agreed terms for Brookfield Asse...
1,Acquisdata Global Industry SnapShot,ACQIND,",i3441,i831,i83107,iblock,ifinal,ifmsoft,iinv,...","aencre,afsvcg,alibab,allpau,altosv,amexpr,amzc...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,gcat,g...",2025-04-21 00:00:00,2025-04-25 10:06:58,2025-04-26 07:33:20.526,",eurz,namz,uk,usa,weurz,",ACQIND0020250425el4l0000b,...,Digital Payments,Issuer Solutions business,Unknown,Unknown,Technology & Payments,Digital Payments,False,0.85,The company is reshaping its portfolio alongsi...,Unknown
2,Acquisdata Global Industry SnapShot,ACQIND,",i3441,i831,i83107,iblock,ifinal,ifmsoft,iinv,...","aencre,afsvcg,alibab,allpau,altosv,amexpr,amzc...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,gcat,g...",2025-04-21 00:00:00,2025-04-25 10:06:58,2025-04-26 07:33:20.526,",eurz,namz,uk,usa,weurz,",ACQIND0020250425el4l0000b,...,Digital Payments,Issuer Solutions business (issuer processing/s...,Issuer Solutions business,US,Technology & Payments,Digital Payments (Issuer Processing),False,0.20,The company is divesting a defined business un...,Global Payments Inc. ... announced definitive ...
3,Acquisdata Global Industry SnapShot,ACQIND,",i3441,i831,i83107,iblock,ifinal,ifmsoft,iinv,...","aencre,afsvcg,alibab,allpau,altosv,amexpr,amzc...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,gcat,g...",2025-04-21 00:00:00,2025-04-25 10:06:58,2025-04-26 07:33:20.526,",eurz,namz,uk,usa,weurz,",ACQIND0020250425el4l0000b,...,Consumer Finance,Unknown,Unknown,Unknown,Unknown,Unknown,False,0.20,"Only a leadership change was announced, with n...","On 23 March 2025, AEON announced appointment o..."
4,Acquisdata Global Industry SnapShot,ACQIND,",i3441,i831,i83107,iblock,ifinal,ifmsoft,iinv,...","aencre,afsvcg,alibab,allpau,altosv,amexpr,amzc...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,gcat,g...",2025-04-21 00:00:00,2025-04-25 10:06:58,2025-04-26 07:33:20.526,",eurz,namz,uk,usa,weurz,",ACQIND0020250425el4l0000b,...,Digital Payments,Separation and possible carve-out of credit ca...,Unknown,CN,Financial Services,Consumer Lending,False,0.20,Regulatory-driven restructuring may force Ant ...,"Ant would also cut any linkage between Alipay,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,GlobalData Company Profiles,GLOMCP,",i8395416,ifinal,","acasij,accacc,bllfxr,bnkmah,cbkire,cnvvu,cofcz...",",c01,c18,c181,cacqu,cactio,ccat,cpartn,ncat,nf...",2025-04-15 00:00:00.000,2009-03-23 09:56:16,2025-04-16 07:32:06.698,",eurz,uk,weurz,",GLOMCP0020250415el4f001yi,...,Fintech,Italian business credit information business,Experian Data Services S.r.l (EDS),IT,Technology & Payments,Fintech,True,0.10,Experian signaled a divestment of its Italian ...,"In March, the company announced its plan to di..."
596,GlobalData Company Profiles,GLOMCP,",i3302,i330202,i3302021,i3302022,i34531,i814,i...","acecoo,akahus,akmt,amdcs,amexpr,artiza,asetea,...",",c01,c18,cactio,ccat,cdiv,cpartn,ncat,nfact,nf...",2025-04-15 00:00:00.000,2008-07-04 12:34:53,2025-04-16 07:40:47.621,",aichi,apacz,asiaz,easiaz,eurz,hyogo,jap,kanag...",GLOMCP0020250415el4f00ch3,...,IT Services,Denmark eDoc Case Management Business,eDoc Case Managemen

In [ ]:
async def search_identify_carveouts(df: pd.DataFrame, concurrency: int = 10) -> pd.DataFrame:
    factory = LLMChainFactory(model_name="o4-mini", provider="azure", streaming=False)
    
    prompt_template = PromptTemplate.from_template(IDENTIFICATION_SEARCH_PROMPT_TEMPLATE)
    runnable = factory.build_search_runnable_with_structured_output(
        pydantic_model=SearchCarveOutIdentificationSummary,
        model="gpt-5"
    )
    chain = prompt_template | runnable

    sem = asyncio.Semaphore(concurrency)

    def should_search(row):
        """Determine if a row should be processed through search"""
        # Only search if potential_disposal_company is NOT "Unknown"
        if row.get("potential_disposal_company", "") == "Unknown":
            return False
        
        # AND if any of these fields are "Unknown"
        unknown_fields = [
            row.get("financial_group_hq", "") in ["Unknown", "N/A", "", "None", "None Identified"],
            row.get("group_industry", "") in ["Unknown", "N/A", "", "None", "None Identified"], 
            row.get("group_vertical", "") in ["Unknown", "N/A", "", "None", "None Identified"],
            row.get("potential_disposal_country", "") in ["Unknown", "N/A", "", "None", "None Identified"],
            row.get("potential_disposal_industry", "") in ["Unknown", "N/A", "", "None", "None Identified"],
            row.get("potential_disposal_vertical", "") in ["Unknown", "N/A", "", "None", "None Identified"],
        ]
        
        return any(unknown_fields)

    async def call_llm(row, pbar):
        async with sem:
            try:
                # Map DataFrame columns to search template variables
                template_data = {
                    "target_company": row.get("target_company", ""),
                    "financial_group": row.get("financial_group", ""),
                    "potential_disposal": row.get("potential_disposal", ""),
                    "potential_disposal_company": row.get("potential_disposal_company", ""),
                    "news_source": row.get("source_name", ""),
                    "article_body": row.get("article_fragment", "")
                }
                
                resp = await chain.ainvoke(template_data)
                
                # Extract parsed model from additional_kwargs
                parsed_model = resp.additional_kwargs['parsed']
                search_results = parsed_model.model_dump()
                
                # Only replace "Unknown" values with search results
                updated_row = row.copy()
                
                # Replace Unknown values with search results
                if row.get("financial_group_hq", "") not in ["Unknown", "N/A", "", "None", "None Identified"]:
                    updated_row["financial_group_hq"] = search_results.get("financial_group_hq", "Unknown")
                
                if row.get("group_industry", "") not in ["Unknown", "N/A", "", "None", "None Identified"]:
                    updated_row["group_industry"] = search_results.get("group_industry", "Unknown")
                    
                if row.get("group_vertical", "") not in ["Unknown", "N/A", "", "None", "None Identified"]:
                    updated_row["group_vertical"] = search_results.get("group_vertical", "Unknown")
                    
                if row.get("potential_disposal_country", "") not in ["Unknown", "N/A", "", "None", "None Identified"]:
                    updated_row["potential_disposal_country"] = search_results.get("potential_disposal_country", "Unknown")
                    
                if row.get("potential_disposal_industry", "") not in ["Unknown", "N/A", "", "None", "None Identified"]:
                    updated_row["potential_disposal_industry"] = search_results.get("potential_disposal_industry", "Unknown")
                    
                if row.get("potential_disposal_vertical", "") not in ["Unknown", "N/A", "", "None", "None Identified"]:
                    updated_row["potential_disposal_vertical"] = search_results.get("potential_disposal_vertical", "Unknown")
                
                # Mark that search was used for this row
                updated_row["search_used"] = True
                
                return updated_row
            finally:
                pbar.update(1)

    # Filter rows that need search and process them
    df_copy = df.copy()
    
    # Initialize search_used column for all rows
    df_copy["search_used"] = False
    
    rows_to_search = []
    rows_unchanged = []
    
    print("Analyzing rows for search requirements...")
    for idx, row in df_copy.iterrows():
        row_dict = row.to_dict()
        row_dict['original_index'] = idx
        
        if should_search(row_dict):
            rows_to_search.append(row_dict)
        else:
            rows_unchanged.append(row_dict)
    
    print(f"Rows requiring search: {len(rows_to_search)}")
    print(f"Rows unchanged: {len(rows_unchanged)}")
    
    # Process search rows with progress tracking
    if rows_to_search:
        with tqdm(total=len(rows_to_search), desc="Processing search identification") as pbar:
            tasks = [asyncio.create_task(call_llm(row, pbar)) for row in rows_to_search]
            search_outputs = await asyncio.gather(*tasks)
        
        # Create DataFrame from search results
        df_searched = pd.DataFrame(search_outputs)
        df_searched.set_index('original_index', inplace=True)
        
        # Update the original DataFrame with search results
        print("Updating original DataFrame with search results...")
        for idx, updated_row in df_searched.iterrows():
            for col in updated_row.index:
                if col != 'original_index':
                    df_copy.loc[idx, col] = updated_row[col]
    else:
        print("No rows require search processing.")
    
    print(f"Search completed. Rows processed with search: {df_copy['search_used'].sum()}")
    
    return df_copy

In [9]:
df_search_output = await search_identify_carveouts(df_output, concurrency=20)
# df_search_output.to_csv("outputs/search_carveout_results.csv", index=False)

Analyzing rows for search requirements...
Rows requiring search: 50
Rows unchanged: 550


Processing search identification:  16%|█▌        | 8/50 [00:59<03:15,  4.66s/it]INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
Processing search identification: 100%|██████████| 50/50 [04:11<00:00,  5.02s/it]

Updating original DataFrame with search results...
Search completed. Rows processed with search: 50


In [6]:
df_search_output.to_csv("outputs/search_carveout_results_2.csv", index=False)

In [11]:
df_search_output[df_search_output["search_used"]]

,source_name,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,date,region_codes,an,...,potential_disposal,potential_disposal_company,potential_disposal_country,potential_disposal_industry,potential_disposal_vertical,relevant,interest_score,rationale,article_quote,search_used
12,Acquisdata Global Industry SnapShot,ACQIND,",i814,i81402,i81404,ibnk,ifinal,ifmsoft,iibnk,...","barc,bkcomc,blkfel,bnkeng,bohbnk,btmln,ccfsgl,...",",c15,c1522,c17,c171,c18,c181,cactio,ccat,cdom,...",2025-02-24 00:00:00,2025-02-26 10:24:31,2025-02-27 07:25:23.238,",eurz,uk,weurz,",ACQIND0020250226el2o00027,...,None identified,N/A,Unknown,Financial Services,Banking,False,0.20,The article points to a completed sale of the ...,Other items of $262m reflect net gain from sal...,True
41,Acquisdata Global Industry SnapShot,ACQIND,",i3302,i330202,i3302021,i81502,i8150203,ialtin...","bafinz,bhuld,bitkmv,blbmsi,bthpqd,cappt,comden...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,ncat,n...",2025-04-02 00:00:00,2025-04-02 10:22:14,2025-04-03 07:31:15.582,",eecz,eurz,gfr,weurz,",ACQIND0020250402el420003f,...,,,DE,Technology & Payments,Software,False,0.10,The article mentions only a completed divestme...,As we work towards a new phase after the sale ...,True
54,Acquisdata Global Industry SnapShot,ACQIND,",i814,i81402,ibnk,ifinal,iibnk,","bafinz,bdis,bnqpst,byshlb,castaq,ciec,combnk,c...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,ncat,n...",2025-01-14 00:00:00,2025-01-15 10:22:41,2025-01-16 07:25:52.211,",eecz,eurz,gfr,weurz,",ACQIND0020250115el1e00016,...,Not specified,Not specified,Unknown,Unknown,Unknown,False,0.30,BayernLB and its subsidiary DKB are pursuing t...,The key factor in the increase at DKB was meas...,True
59,Acquisdata Global Industry SnapShot,ACQIND,",i814,i81402,i82,ibnk,ifinal,","allanz,allass,andac,aszngn,banqnp,bnps,borsit,...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,ncat,n...",2025-04-16 00:00:00,2025-04-16 10:22:46,2025-04-17 07:32:19.703,",eecz,emilia,eurz,italy,ligur,medz,piedm,weurz,",ACQIND0020250416el4g0003d,...,None identified,N/A,IT,Financial Services,Unknown,False,0.00,The article provides a corporate overview and ...,Poste Italiane is the largest integrated and o...,True
75,Acquisdata Global Industry SnapShot,ACQIND,",i814,i81402,ibnk,ifinal,","bbv,bisett,blfima,ceg,crdts,deut,dunbst,efgisw...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,ncat,n...",2025-03-17 00:00:00,2025-03-19 10:22:30,2025-03-20 07:27:01.796,",eurz,switz,weurz,",ACQIND0020250319el3h00012,...,N/A,N/A,Unknown,Unknown,Unknown,False,0.10,The article emphasizes long-term investment an...,"As we stay close to clients, helping them navi...",True
79,Acquisdata Global Industry SnapShot,ACQIND,",i81502,i8150203,i82,i82002,ialtinv,ifinal,iinv,","acordc,aiu,assbn,blaman,bnkeng,bpad,chesn,clif...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,ncat,n...",2025-03-18 00:00:00,2025-03-19 10:24:40,2025-03-20 07:28:55.419,",eurz,uk,weurz,",ACQIND0020250319el3i0003p,...,"None announced; prior sales (Cala, US Protecti...",N/A,Unknown,Unknown,Unknown,False,0.20,The article highlights completed divestments (...,By sharpening our focus and simplifying our po...,True
90,Acquisdata Global Industry SnapShot,ACQIND,",i81502,i8150203,i82,i82002,ialtinv,ifinal,iinv,","acordc,aiu,assbn,blaman,bnkeng,bpad,chesn,clif...",",c18,c181,cacqu,cactio,ccat,cdom,ciprof,ncat,n...",2025-03-18 00:00:00,2025-03-19 10:24:40,2025-03-20 07:28:55.419,",eurz,uk,weurz,",ACQIND0020250319el3i0003p,...,None announced,N/A,Unknown,Unknown,Unknown,False,0.20,The article focuses on cost-saving restructuri...,We also collapsed our Heritage and Open divisi...,True
98,Business News Africa,AFNBU,",i814,i81402,i8396,ibcs,ibnk,ifinal,","accbnk,curmot,jobgse,mogabn",",c18,cactio,ccat,cdiv,ncat,nfact,nfcpin,",2024-12-13 13:31:18,2024-12-13 13:35:06,2024-12-14 07:12:30.320,",africaz,devgcoz,dvpcoz,eurz,nigea,safr,souafr...",AFNBU00020241213ekcd0008d,...,N/A,N/A,Unknown,Unknown,Unknown,False,0.00,The 

In [17]:
df1 = pd.read_csv("/Users/sylvestrsemesko/Desktop/search_carveout_results.csv")

In [24]:
df = df1[(df1['potential_disposal_country_x'] != df1['potential_disposal_country_y']) & (df1['potential_disposal_country_x'] != "Unknown") & (df1['potential_disposal_country_y'] != "Unknown")]

In [25]:
len(df)

46

In [26]:
df.to_csv("outputs/diff.csv", index=False)

In [27]:
df = pd.read_csv("/Users/sylvestrsemesko/Downloads/preprocessing_result.csv")

In [28]:
df

,source_name,title,snippet,body,section,word_count,source_code,industry_codes,company_codes,subject_codes,publication_datetime,ingestion_datetime,modification_date,region_codes,an,action,article,company_names,industries
0,Interfax: Russia & CIS Business and Financial ...,"Daily Headline News for March 14, 2025",Digest of headline news as of 7:00 p.m. Moscow...,*** RUSSIA'S BROAD MONETARY BASE DECLINES 2.8%...,NaN,1990,DAIBN,",i1,i111,i22,i224,i2245,ibasicm,ibnk,icoal,ifi...",rosgos,",c18,c181,c33,c333,cacqu,cactio,ccat,ncat,ncdi...",2025-03-14 16:00:00.000,2025-03-14 18:00:07,2025-03-15 07:24:00.694,",asiaz,eeurz,eurz,russ,",DAIBN00020250314el3e006y1,add,"Daily Headline News for March 14, 2025\n*** RU...",Central Bank of the Russian Federation,"Energy,Coal Mining,Primary Metals,Non-ferrous ..."
1,M2 Presswire,Impending fintech consolidation could yield un...,The fintech sector is sustaining its rapid exp...,"According to Victor Basta, Managing Partner at...",NaN,621,MTPW,",ifinal,ifmsoft,itech,",NaN,",c18,c181,cacqu,cactio,ccat,ncat,nfact,nfcpin,...",2025-01-28 00:00:00.000,2025-01-28 09:07:17,2025-01-29 07:25:31.631,",eurz,uk,weurz,",MTPW000020250128el1s00001,add,Impending fintech consolidation could yield un...,NaN,"Financial Services,Financial Technology,Techno..."
2,Financial News,UK's private share trading plan will be a flop...,The UK's plan to allow trading of private comp...,The Private Intermittent Securities and Capita...,Private Equity,717,LONFIN,",i81502,i8150203,ialtinv,ifinal,iinv,",bttcap,",c13,c18,c181,cacqu,cactio,ccat,ncat,nfact,nfc...",2025-03-06 00:00:00.000,2025-03-06 00:00:07,2025-03-10 07:18:14.212,",eurz,uk,weurz,",LONFIN0020250306el3600001,add,UK's private share trading plan will be a flop...,Better Capital LLP,"Trusts/Funds/Financial Vehicles,Private Equity..."
3,MFN,VEF initiates a new share buyback program,VEF AB (publ) (“VEF” or the “Company”) has rec...,"Dave Nangle, CEO of VEF comments:\n\n“VEF shar...",NaN,442,MFNEN,",i81502,i8150203,ialtinv,ifinal,iinv,iventure,",zomhfn,",c17,c171,c18,c181,c41,cactio,cboard,ccat,cshb...",2025-04-11 00:00:00.000,2025-04-11 06:00:04,2025-04-12 07:24:33.453,",eecz,eurz,nordz,stock,swed,",MFNEN00020250411el4b0005m,add,VEF initiates a new share buyback program\nDav...,VEF AB,"Trusts/Funds/Financial Vehicles,Private Equity..."
4,CE NoticiasFinancieras,Factop case: Swiss investment fund sues Sauer ...,Nearly two years after the outbreak of the so-...,The vehicle claims to have been defrauded by t...,CE Noticias Financieras English,665,NFINCE,",i81502,ifinal,iinv,",NaN,",c12,c18,c33,c333,cactio,ccat,cliqui,gcat,gcri...",2025-02-18 00:00:00.000,2025-02-18 17:20:30,2025-02-19 07:28:25.421,",eurz,switz,weurz,",NFINCE0020250218el2i008tb,add,Factop case: Swiss investment fund sues Sauer ...,NaN,"Trusts/Funds/Financial Vehicles,Financial Serv..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2906,Public Companies News and Documents via PUBT,Vinci SA - Determination of the share referenc...,Access the original document here\n\nDetermina...,"Nanterre, 18 February 2025\n\nDetermination of...",NaN,2444,LCDVP,",i831,iconst,icre,ifinal,iinv,","adesmf,banqnp,barbip,crdnat,dwitd,euecar,eurun...",",c17,c172,cactio,ccat,ncat,nfact,nfcpin,npress,",2025-02-18 00:00:00.000,2025-02-18 17:49:17,2025-02-19 07:29:07.855,",eecz,eurz,fra,medz,namz,uk,usa,weurz,",LCDVP00020250218el2i00tbo,add,Vinci SA - Determination of the share referenc...,"Autorité des Marchés Financiers (France),BNP P...","Financial Investment Services,Construction,Rea..."
2907,Public Companies News and Documents via PUBT,Vinpai SA - Details of the Terms and Condition...,Access the original document here\n\nDetails o...,* Camlin Fine Science Limited and certain sign...,NaN,2333,LCDVP,",i25,i41,i831,i83106,ibasicm,icnp,ifinal,ifood...","adesmf,bobexh,camfcm,eparlm,euecar,nseoil,scex...",",c18,c181,cacqu,cactio,ccat,ctendo,ncat,nfact,...",2025-03-13 00:00:00.000,2025-03-13 16:52:33,2025-03-14